# Week 06

## Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024S-R/9103-utils/raw/main/src/io_utils.py
!wget -q https://github.com/DM-GY-9103-2024S-R/9103-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

from data_utils import MinMaxScaler, StandardScaler
from io_utils import object_from_json_url

## Dataset Exploration

Let's revisit the house dataset from last homework.

In [ ]:
# Define the location of the json file here
HOUSES_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/LA_housing.json"

houses_info = object_from_json_url(HOUSES_FILE)

### Some "light" exploration:

Ok. We should now have a list of objects with information about houses in LA.

Let's work with the data to answer the following questions:
- How many houses are in this dataset?
- How many *features* does our dataset have?
- What are the *features*?
- What's the min, max and average price for the houses in the dataset?

In [ ]:
# How many houses are in the dataset?
num_houses = len(houses_info)

# Features:
house_features = houses_info[0].keys()

# Number of features:
num_features = len(house_features)

houses_info[0], house_features, num_houses, num_features

In [ ]:
# List of prices

house_prices = []
for h in houses_info:
  house_prices.append(h["value"])

# min price
min_price = min(house_prices)

# max price
max_price = max(house_prices)

# avg price
avg_price = sum(house_prices) // num_houses

min_price, max_price, avg_price

### More exploring

What if we wanted to get `min`, `max` and `average` values for all of the features?

# 😖

Repeating the code above, can get really annoying really quick, but hopefully we can use the `Pandas` library to help.

Once we load our data into a `DataFrame` we can perform many types of calculations.

Here's how we load our data into a `DataFrame`:

In [ ]:
houses_df = pd.DataFrame.from_records(houses_info)

# And to check the first couple of rows/records
houses_df.head()

### DataFrame

<img src="./imgs/data_frame.jpg" width="700px">

Now we have access to each feature by simply indexing the `DataFrame` by the feature's name.

For example, to get a list of all of the prices:

In [ ]:
house_prices = houses_df["value"]

# only print first 5
house_prices.head()

Or a list of all of the ages of the houses:

In [ ]:
house_ages = houses_df["age"]

# only print first 5
house_ages.head()

Or, we can get a sub-section of our original data with only the `age` and `value` features:

In [ ]:
house_ages_values = houses_df[["age", "value"]]

# only print first 5
house_ages_values.head()

### Doing some maths

To get the smallest value of a *feature*:

In [ ]:
min_price = houses_df["value"].min()
min_price

And the `average` for a *feature*:

In [ ]:
avg_price = houses_df["value"].mean()
avg_price

This gives us a list with the names of the *features*/*columns*:

In [ ]:
house_features = list(houses_df.columns)
house_features

It's similar to when we did `houses_info[0].keys()` above.

Either way, we can now iterate over a list of the feature names to calculate `min`, `max` and `average` for each *feature*:

In [ ]:
for f in house_features:
  print(f)
  print("\tmin:", houses_df[f].min())
  print("\tmax:", houses_df[f].max())
  print("\tavg:", houses_df[f].mean())

We can even ask for the `min` (or `max`, or `average`) of the whole `DataFrame` and it knows to do it for each column:

In [ ]:
houses_df.min()

### Average / Mean

The average, or mean, value of a set of numbers is a quantity that represents the center of a collection of numbers. What this means is that we expect about half of the numbers in a collection to be higher than the mean, and the other half to be lower.

The mean of a set of numbers $x_1, x_2, ..., x_n$ is calculated by dividing the sum of the values by the number of values. It's sometimes written like this:

$\displaystyle \mu = \frac{1}{n} \sum_{i=1}^{n} x_i$

which is the same as `sum(X) / len(X)` in Python if `X` is our list of values.

### Standard Deviation

In addition to the mean, the standard deviation is a measure of the amount of variation in a sequence of numbers.

It's calculated by taking the square root of the average of the squared differences of each point to the mean of the sequence.

In other words, first we calculate the difference between each point and the mean, and square this difference, then sum all of them up, divide by the number of values in the sequence, and finally take the square root:

$\displaystyle \sigma = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(x_{i} - \mu \right)^{2}}$

The standard deviation is a measurement of how close all of the points are to the mean.

<img src="./imgs/std_dev.jpg" width="800px"/>

Unfortunately, there is no short Python code for computing the standard deviation like there is for the average, but the `DataFrame` object has a function for computing it:

In [ ]:
# mean and standard deviation for each feature
for f in houses_df.columns:
  print(f)
  print("\tavg:", houses_df[f].mean())
  print("\tstd:", houses_df[f].std())

We can see that there's greater variability in the number of rooms of a house when compared to the number of bedrooms, but besides that, this isn't very useful yet because we can't really compare the standard deviations from different *features* that have different units.

We'll see soon how we can use `mean` and `standard deviation` to be able to compare, combine, extrapolate values that were measured in different units.

### Correlation

Let's say we want to figure out if there are any *features* that correlate strongly with the house prices.

This means figuring out if there are any other *features* that are a good indication for the value of a house.

We only have a handful of *features*, so we can always plot them:

In [ ]:
# A lot easier with Pandas DataFrames

for f in house_features:
  if f != "value":
    plt.scatter(houses_df[f], houses_df["value"], alpha=.1)
    plt.xlabel(f)
    plt.ylabel("value")
    plt.show()

### Covariance Matrix

Instead of looking at the plots there's actually a mathematical way of calculating how much the *features* of a dataset are *related*.

It's called the [covariance](https://en.wikipedia.org/wiki/Covariance), and it measures how much $2$ *features* change together.

And with our `DataFrame` we can just call:

In [ ]:
houses_df.cov()

This shows us how much each variable is related to every other variable, which will be useful when we start training models and want to reduce the amount of data we need to process, but for now we can just look at the covariances for the `value` *feature*:

In [ ]:
houses_df.cov()["value"]

If we look at the *covariances* with the largest magnitudes, we can see that:
- `value` correlates with `rooms`: the more rooms, the higher the price of a house.
- `value` correlates inversely with `age`: the older the house, the less valuable it is.
- `value` correlates somewhat with `longitude`.

### Normalizing / Standardizing / Scaling

The above calculation gives us some extra information about the data, but there's a problem:

All of the *features* are in different units. The range of the `rooms` *feature* is between $1$ and $17$, while the range of `age` is $2$ to $55$, and `latitude` and `longitude` vary by at most $1$ degree.

Using different units to calculate the *covariance* can exaggerate how much certain *features* actually influence each other.

In order to calculate *covariance* correctly we have to normalize the data in all columns to be *unitless*.

One way to do this is to scale each value to be within the range $[0, 1]$ relative to the `min` and `max` values of their column.

### [MinMax Scaling](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html):

<img src="./imgs/scaling_min_max.jpg" width="700px">

The math is not very complicated, it's actually exactly like the `map()` function from `p5.js`, but we're going to use a library that will do this for us.

In [ ]:
# Scale all the values to be between 0, 1

# This creates a scaler object we can use
min_max_scaler = MinMaxScaler()

# This is how we use the object on our data
houses_min_max_df = min_max_scaler.fit_transform(houses_df)

# Scaled version of DataFrame
houses_min_max_df.head()

In [ ]:
# Now we can finally correctly calculate the covariances
houses_min_max_df.cov()["value"]

After we scale our features according to the `min` and `max` value of each column, `longitude` is the top feature that correlate with `value`, followed by `rooms` and then `age`.

### [Standard Scaling](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

This is another type of *normalization* that uses the `mean` and `standard deviation` for each column to center their values at $0$ and have a range of about $[-3, 3]$.

<img src="./imgs/scaling_std.jpg" width="700px">

The math is a bit more involved, but luckily this *scaling* is also available from the [Scikit-Learn](https://scikit-learn.org) library:

In [ ]:
# Scale all the values to be centered at 0 and (mostly) within [-3, 3]

# This creates a scaler object we can use
std_scaler = StandardScaler()

# This is how we use the object on our data
houses_std_df = std_scaler.fit_transform(houses_df)

# Scaled version of DataFrame
houses_std_df.head()

In [ ]:
# Now we can finally correctly calculate the covariances
houses_std_df.cov()["value"]

We can see that if we scale our features according to the `mean` and `std dev` of each column, `longitude` is the top feature that correlate with `value`, followed by `rooms` and then `age`.

In this case `MinMax` and `Standard` *scaling* are in agreement, but it's not always the case.

Either way, *Normalizing* (or, *Standardizing*, or, *Scaling*) is an important step when working with data.

This is not only true for calculating *covariances* and learning something about our datasets, but is even more important when training models with multiple *features*.

## One more dataset

Let's load...

In [ ]:
# Define the location of the json file here
DIAMONDS_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/diamonds.json"

# Read into Python object
diamonds_data = object_from_json_url(DIAMONDS_FILE)

# Create DataFrame
diamonds_df = pd.DataFrame.from_records(diamonds_data)

diamonds_df.head()

In [ ]:
for f in diamonds_df.columns:
  print(f)
  print("\tmin:", diamonds_df[f].min())
  print("\tmax:", diamonds_df[f].max())
  print("\tavg:", diamonds_df[f].mean())
  print("\tstd:", diamonds_df[f].std())

# 😫

Some of the *features* aren't even numerical, they're words/tags that describe some property of the diamond.

We can still do some analysis. The easiest way is to just drop those columns:

In [ ]:
diamonds_numerical_df = diamonds_df.drop(columns=["cut", "color", "clarity"])
diamonds_numerical_df.head()

### Now we can get some statistics

In [ ]:
for f in diamonds_numerical_df.columns:
  print(f)
  print("\tmin:", diamonds_numerical_df[f].min())
  print("\tmax:", diamonds_numerical_df[f].max())
  print("\tavg:", diamonds_numerical_df[f].mean())
  print("\tstd:", diamonds_numerical_df[f].std())

# 🤷

Which is fine if all we're doing is taking a look at our data, but once we start training models and doing more detailed analyses we'll want to use *ALL* of the data that we have available.

Let's see how to work with non-numerical properties.

### A Note About Feature

First, let's think about and organize the different types of *features* we have seen so far.

We have some *features* that are represented by *continuous* values, like: `price` or `length`.

We also have *features* that are represented by *ordered* discrete values, like: `clothing size` $(S, M, L)$, or `letter grades` $(A, B, C, D, F)$.

And, we can also have *features* that are represented by *UNordered* discrete values (sometimes called *categories*), like: `primary colors` $(red, green, black)$, or `states` $(PA, NY, NJ)$.

<img src="./imgs/encoding_00.jpg" width="700px">

### Encoding

In order to analyze datasets that have different types of *features*, it helps to be able to represent the *categorical* and *discrete* *features* in terms of numbers.

This way we can perform the same types of transformations and calculations, like `mean`, `standard deviation` and `covariances`, on all of the *features* together.

We'll achieve this by *encoding* these *features* using numbers. We'll create associations between the *feature* values and specific numbers, so we can easily transform these *categories* into numbers, do math, and then un-transform them back to *categories*.

### Ordinal Encoding

If the data that we want to encode has some kind of order, like with `letter grades`, where $A$ comes before $B$, which comes before $C$, and so on, we can use something called an [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html).

We provide the order of the possible values of the *feature* and they will each get represented by a number in that same order:

<img src="./imgs/encoding_ordinal.jpg" width="700px"/>

### One-Hot Encoding

If, however, the values we want to encode don't really have an oder, like `states`, or even `zip codes` (these are numbers, but it's not like we have a good reason for considering $10001$ *smaller* than $15201$), we can use an [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

This encoder turns each of the possible values of a *feature* into its own *feature*, and uses the numbers $0$ and $1$ to mark whether a *record* had that value or not:

<img src="./imgs/encoding_one_hot.jpg" width="700px"/>

### Encoding Diamonds

If we look at the [dataset info](https://www.kaggle.com/datasets/shivam2503/diamonds) for our diamond data, we'll see that all three of our *discrete* *features* have an order:

<img src="./imgs/diamonds.jpg" width="700px"/>

They all have a *worse* and a *best* option.

This means we can use an [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) to assign numbers to them and include them back into our dataset analysis and eventually use them for training models.

To use the encoder, we first we have to determine all the possible values of each of these features:

In [ ]:
for f in ["cut", "color", "clarity"]:
  print(diamonds_df[f].unique())

Then re-order them from worst to best:

In [ ]:
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

Create the encoder:

In [ ]:
diamond_encoder = OrdinalEncoder(categories=[cut_order, color_order, clarity_order])

And apply it to our `DataFrame`:

In [ ]:
# Encode the columns
ccc_vals = diamond_encoder.fit_transform(diamonds_df[["cut", "color", "clarity"]].values)

# Put the values back in the original DataFrame
diamonds_df[["cut", "color", "clarity"]] = ccc_vals

Now we can rerun our loop to get statistics:

In [ ]:
for f in diamonds_df.columns:
  print(f)
  print("\tmin:", diamonds_df[f].min())
  print("\tmax:", diamonds_df[f].max())
  print("\tavg:", diamonds_df[f].mean())
  print("\tstd:", diamonds_df[f].std())

Normalize the data, get covariances and plots

In [ ]:
# Scale all the values to be between 0, 1

# This creates a scaler object we can use
min_max_scaler = MinMaxScaler()

# This is how we use the object on our data
diamonds_min_max_df = min_max_scaler.fit_transform(diamonds_df)

# Scaled version of DataFrame
diamonds_min_max_df.head()

In [ ]:
diamonds_min_max_df.cov()["price"]

In [ ]:
# Plot the original data
for f in diamonds_df.columns:
  if f != "price":
    plt.scatter(diamonds_df[f], diamonds_df["price"], alpha=.1)
    plt.xlabel(f)
    plt.ylabel("price")
    plt.show()

## Next time: Regression

In [ ]:
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

from data_utils import LinearRegression, MinMaxScaler
from io_utils import object_from_json_url

# Load Dataset
DIAMONDS_FILE = "https://raw.githubusercontent.com/DM-GY-9103-2024S-R/9103-utils/main/datasets/json/diamonds.json"

# Read into DataFrame
diamonds_data = object_from_json_url(DIAMONDS_FILE)
diamonds_df = pd.DataFrame.from_records(diamonds_data)


# Encode
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']
clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

diamond_encoder = OrdinalEncoder(categories=[cut_order, color_order, clarity_order])

ccc_vals = diamond_encoder.fit_transform(diamonds_df[["cut", "color", "clarity"]].values)
diamonds_df[["cut", "color", "clarity"]] = ccc_vals


# Normalize
diamond_scaler = MinMaxScaler()
diamonds_scaled = diamond_scaler.fit_transform(diamonds_df)


# Linear Regression
price_model = LinearRegression()

# Get prices and features
prices = diamonds_scaled["price"]
features = diamonds_scaled.drop(columns=["price"])

# Build the model
price_model.fit(features, prices)

# Run the model on the training data
predicted_scaled = price_model.predict(features)


# Un-normalize the data
predicted = diamond_scaler.inverse_transform(predicted_scaled)


# Measure Model error
price_mse = mean_squared_error(diamonds_df["price"].values, predicted["price"].values, squared=False)
print(price_mse)


# Plot predictions
carats = diamonds_scaled[["carat"]]

plt.plot(sorted(prices), marker='o', markersize=8)
plt.plot(sorted(predicted_scaled["price"]), marker='o', markersize=2, linestyle='', color='r')
plt.title("sorted prices")
plt.show()

plt.scatter(carats, prices, marker='o', linestyle='', alpha=0.3)
plt.scatter(carats, predicted_scaled, color='r', marker='o', linestyle='', alpha=0.1)
plt.xlabel("carat")
plt.ylabel("price")
plt.show()